# Text Similarity

## Initialize

### Load libraries

In [4]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
from datasets import load_dataset
import tensorflow as tf
from gensim.models import fasttext
from textsim_class import TextSimilarity
import spacy
from gensim.models import KeyedVectors
from gensim.models import word2vec
import pandas as pd
from transformers import AutoTokenizer, AutoModel

### Load models

In [2]:
roberta_tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
roberta_model = AutoModel.from_pretrained('projecte-aina/roberta-base-ca-v2')
roberta_model.eval()
print("done")
model_finetuned = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer_finetuned = AutoTokenizer.from_pretrained(model_finetuned)
model_finetuned = AutoModel.from_pretrained(model_finetuned)

Some weights of RobertaModel were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


done


Some weights of RobertaModel were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2-cased-sts and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
dataset = load_dataset("projecte-aina/sts-ca", trust_remote_code=True)

In [1]:
"""
from gensim.models import FastText
import gensim
# No funciona de moment
model = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')
"""
pass

In [4]:
wv_model = word2vec.Word2Vec.load(f"./models/word2vec_10000_300_10_5.model")
wv_model = wv_model.wv

In [5]:
wv_model_1000 = word2vec.Word2Vec.load(f"./models/cai/w2v_1000.model")
wv_model_1000 = wv_model_1000.wv

In [6]:
wv_model_500 = word2vec.Word2Vec.load(f"./models/cai/w2v_500.model")
wv_model_500 = wv_model_500.wv

In [7]:
wv_model_100 = word2vec.Word2Vec.load(f"./models/word2vec_100_3.model")
wv_model_100 = wv_model_100.wv

In [8]:
from gensim.models import KeyedVectors
# Word2vec permet dos formats: text i binari
kv_model = KeyedVectors.load_word2vec_format('./models/word2vec/34/model.bin', binary=True)

In [ ]:
ft_model = fasttext.load_facebook_model('./models/fasttext/cc.ca.300.bin.gz')
ft_model = ft_model.wv
"""
Doesn't work
from gensim.models import FastText
ft_model = FastText.load('./models/fasttext/cc_ca_300.bin', mmap='r')
"""
pass

In [10]:
model_spacy = spacy.load("ca_core_news_md")

In [11]:
model_roberta = spacy.load("ca_core_news_trf")

## Execute all the models

In [12]:
list_of_models = [
    # model     mode    pretr   cls remap   train   recalc  name
    (wv_model, "onehot", True, True, True, True,True,  "Onehot"),
    (wv_model, "mean",  True, True, True, True, True, "Word2vec propi mean"),
    (wv_model, "tfidf",  True, True, True, True, True,"Word2vec propi tfidf"),
    (wv_model_1000, "mean",  True, True, True, True, True, "Word2vec 1000 mean"),
    (wv_model_1000, "tfidf",  True, True, True, True, True,"Word2vec 1000 tfidf"),
    (wv_model_500, "mean",  True, True, True, True, True, "Word2vec 500 mean"),
    (wv_model_500, "tfidf",  True, True, True, True, True,"Word2vec 500 tfidf"),
    (wv_model_100, "mean",  True, True, True, True, True, "Word2vec 100 mean"),
    (wv_model_100, "tfidf",  True, True, True, True, True,"Word2vec 100 tfidf"),
    (kv_model, "mean",  True, True, True, True, True,"Word2vec nlpl mean"),
    (kv_model, "tfidf",  True, True, True, True, True,"Word2vec nlpl tfidf"),
    (ft_model, "mean",  True, True, True, True, True,"Word2vec ccca300 mean"),
    (ft_model, "tfidf",  True, True, True, True, True,"Word2vec ccca300 tfidf"),
    (model_spacy, "spacy",  True, True, True, True, True, "Spacy md"),
    (model_roberta, "roberta", True, False, True, True, "roberta_mean_mapped_pairs", "Roberta mean"),
    (model_roberta, "roberta", True, True, True, True, "roberta_mapped_pairs", "Roberta cls"),
    (model_roberta, "roberta2", True, False, True, True, "roberta2_mapped_pairs", "Roberta 0 mean"),
    (model_roberta, "roberta2", True, True, True, True, "roberta2_cls_mapped_pairs", "Roberta 0 cls"),
    (roberta_model, "roberta-hugging", True, False, True, True, "roberta-hugging_mean_mapped_pairs", "Roberta hugging mean"),
    (roberta_model, "roberta-hugging", True, True, True, True, "roberta-hugging_cls_mapped_pairs", "Roberta hugging cls"),
    (model_finetuned, "roberta-hugging", True, False, True, True, "roberta_finetuned_mean_mapped_pairs", "Roberta finetuned mean"),
    (model_finetuned, "roberta-hugging", True, True, True, True, "roberta_finetuned_cls_mapped_pairs", "Roberta finetuned cls"),
    (wv_model, "embeddings", False, True, True, False, True, "Random v2"),
    (wv_model, "embeddings", False, True, True, True, True, "Random"),
    (wv_model, "embeddings", True, True, True, False, True, "Initialized w2v propi notrain"),
    (wv_model, "embeddings", True, True, True, True, True, "Initialized w2v propi"),
    (ft_model, "embeddings", True, True, True, False,True,  "Initialized ccca300 notrain"),
    (ft_model, "embeddings", True, True, True, True, True, "Initialized ccca300")
]

In [13]:
results_dict: dict[str, dict[str, str|float|None]] = {
    model_name + str(exec_model): {
        'Model': model_name,
        'Exec model': exec_model,
        'Mode': None, 
        'Pretrained': None, 
        'CLS': None, 
        'Trained': None, 
        'Train pearson': None, 
        'Val pearson': None,
        'Test pearson': None,
        'Train spearman': None, 
        'Val spearman': None,
        'Test spearman': None
        
        } for model_name in [l[-1] for l in list_of_models] for exec_model in range(9)
}

In [14]:
total_models = len(list_of_models)
count = 0

Be careful, it takes more than 1 hour

In [15]:
for information in list_of_models:
    model, mode, pretrained, cls, remap, trainable, recalculate, name = information

    text_sim = TextSimilarity(model=model, dataset=dataset, mode=mode, pretrained=pretrained, cls=cls, remap=remap, trainable=trainable, recalculate=recalculate, dict_size=1000000)

    text_sim.define_model(id=0)
    train_pearson, val_pearson, test_pearson, train_spearman, val_spearman, test_spearman = text_sim.baseline_model()



    results_dict[name + str(8)] = {
                'Model': name,
            'Exec model': "Baseline",
            'Mode': mode, 
            'Pretrained': pretrained, 
            'CLS': cls, 
            'Trained': trainable, 
            'Train pearson': train_pearson, 
            'Val pearson': val_pearson,
            'Test pearson': test_pearson,
            'Train spearman': train_spearman, 
            'Val spearman': val_spearman,
            'Test spearman': test_spearman
        }
    
    for index in range(8):
        text_sim.define_model(id=index)
        train_pearson, val_pearson, test_pearson, train_spearman, val_spearman, test_spearman = text_sim.train(num_epochs=128)
        results_dict[name + str(index)] = {
                'Model': name,
            'Exec model': str(index),
            'Mode': mode, 
            'Pretrained': pretrained, 
            'CLS': cls, 
            'Trained': trainable, 
            'Train pearson': train_pearson, 
            'Val pearson': val_pearson,
            'Test pearson': test_pearson,
            'Train spearman': train_spearman, 
            'Val spearman': val_spearman,
            'Test spearman': test_spearman
        }
        print(f"Tensorflow model {index+1}/8 done")
    count += 1
    print(f"{count} models done of a total of {total_models}")

Tensorflow model 1/8 done

Tensorflow model 2/8 done
Tensorflow model 3/8 done
Tensorflow model 4/8 done
Tensorflow model 5/8 done
Tensorflow model 6/8 done
Tensorflow model 7/8 done
Tensorflow model 8/8 done
1 models done of a total of 28
Tensorflow model 1/8 done
Tensorflow model 2/8 done
Tensorflow model 3/8 done
Tensorflow model 4/8 done
Tensorflow model 5/8 done
Tensorflow model 6/8 done
Tensorflow model 7/8 done
Tensorflow model 8/8 done
2 models done of a total of 28
Tensorflow model 1/8 done
Tensorflow model 2/8 done
Tensorflow model 3/8 done
Tensorflow model 4/8 done
Tensorflow model 5/8 done
Tensorflow model 6/8 done
Tensorflow model 7/8 done
Tensorflow model 8/8 done
3 models done of a total of 28
Tensorflow model 1/8 done
Tensorflow model 2/8 done
Tensorflow model 3/8 done
Tensorflow model 4/8 done
Tensorflow model 5/8 done
Tensorflow model 6/8 done
Tensorflow model 7/8 done
Tensorflow model 8/8 done
4 models done of a total of 28
Tensorflow model 1/8 done
Tensorflow model 

Save the results

In [16]:
results_df = pd.DataFrame(results_dict).T

# Reorder the columns and sort the values by the 'Avg. Statistic' column
results_df = results_df[['Model', 'Exec model', 'Mode', 'Pretrained', 'CLS', 'Trained', 'Train pearson', 'Val pearson', 'Test pearson', 'Train spearman', 'Val spearman', 'Test spearman']]
results_df = results_df.sort_values(by='Test pearson', ascending=False)


results_df.to_csv('./results/similarity_results.csv', index=False)
print(f"Results saved to './results/similarity_results.csv'.")

Results saved to './results/similarity_results.csv'.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def generate_model_barplot(results_df: pd.DataFrame, metric: str, model = "all") -> None:
	"""
	Generate a barplot of the models evaluated for the given metric.

	Parameters
	----------
	results_df : pd.DataFrame
		The DataFrame containing the results of the evaluation.
	metric : str
		The metric to generate the barplot for. It can be one of ['Avg. Statistic', 'Pearson', 'Spearman', 'OOV Perc.'].
	family : str, optional
		The family of models to generate the barplot for. It can be one of ['all', 'w2v', 'ft']. The default is 'all'.
	"""
	assert metric in ['Train pearson', 'Val pearson', 'Test pearson'], f"Invalid metric '{metric}'. Must be one of ['Avg. Statistic', 'Pearson', 'Spearman', 'OOV Perc.']."
	# Filter the dataframe by the family of models

	if model != "all":
		results_filtered_df = results_df[results_df['Exec model']==model].copy()
	else:
		results_filtered_df = results_df.copy()

	# Sort the values by the metric (descending order, except for 'OOV Perc.')
	results_filtered_df = results_filtered_df.sort_values(by=metric, ascending=(metric == 'OOV Perc.'))

	# Remove .model from the model names
	results_filtered_df['Model'] = results_filtered_df['Model'].apply(lambda x: x.removesuffix('.model'))

	# Generate the barplot
	plt.figure(figsize=(10, 6))
	sns.barplot(x='Model', y=metric, data=results_filtered_df)
	plt.xticks(rotation=45, ha='right')
	plt.xlabel('Model')
	plt.ylabel(metric)
	plt.tight_layout()
	plt.savefig(f'./results/plots/textsim_{model}_{metric.lower().replace(".", "").replace(" ", "_")}.png')
	plt.close()

generate_model_barplot(results_df, "Test pearson", model="Baseline")

## Single model tests

In [25]:
from importlib import reload

In [171]:
import textsim_class
reload(textsim_class)
import textsim_class

In [ ]:
from transformers import AutoTokenizer, AutoModel

roberta_tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
roberta_model = AutoModel.from_pretrained('projecte-aina/roberta-base-ca-v2')
roberta_model.eval()
print("done")
model_finetuned = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer_finetuned = AutoTokenizer.from_pretrained(model_finetuned)
model_finetuned = AutoModel.from_pretrained(model_finetuned)
model_finetuned.eval()
pass

In [172]:
a = textsim_class.TextSimilarity(model=wv_model, dataset=dataset, mode="mean", pretrained=False, cls=True, remap=True, trainable=True, tokenizer = tokenizer_finetuned, dict_size=1000000, recalculate=True)

In [173]:
a.define_model(id=7)

In [164]:
print(a.exec_model.summary())

Model: "functional_997"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_998     │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_999     │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2161 (Dense)  │ (None, 768)       │    590,592 │ input_layer_998[… │
│                     │                   │            │ input_layer_999[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_757 (Lambda) │ (None, 768)       │          0 │ dense_2161[0][0], │
│                     │                   │            │ dense_2161[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1291        │ (None, 768)       │          0 │ lambda_757[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2162 (Dense)  │ (None, 16)        │     12,304 │ dropout_1291[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_2162[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1292        │ (None, 16)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2163 (Dense)  │ (None, 20)        │        340 │ dropout_1292[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20)        │         80 │ dense_2163[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1293        │ (None, 20)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2164 (Dense)  │ (None, 16)        │        336 │ dropout_1293[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_2164[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1294        │ (None, 16)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2165 (Dense)  │ (None, 1)         │         17 │ dropout_1294[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_758 (Lambda) │ (None, 1)         │          0 │ dense_2165[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 603,797 (2.30 MB)

 Trainable params: 603,693 (2.30 MB)

 Non-trainable params: 104 (416.00 B)

None


In [174]:
pearson_train, pearson_val, pearson_test = a.train(num_epochs=128)

In [175]:
print(pearson_train, pearson_val, pearson_test )

0.757942345458769 0.4161434099970364 0.46045058558859026


In [ ]:
from scipy.stats import pearsonr
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = a.exec_model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
print(f"Correlación de Pearson (baseline-train): {compute_pearson(a.x_train, a.y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson(a.x_val, a.y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson(a.x_test, a.y_test)}")



65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (baseline-train): 0.3534391395435749
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-validation): 0.07699755710073927
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-test): 0.062160707942908074


# Finetuned model

In [1]:
from scipy.stats import pearsonr, spearmanr

In [52]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)


[{'label': 'SIMILARITY', 'score': 2.1183004307689126}, {'label': 'SIMILARITY', 'score': 2.179974932297432}, {'label': 'SIMILARITY', 'score': 0.9511617858568939}]


In [39]:
input_pairs = [(e["sentence1"], e["sentence2"]) for e in dataset["train"]]
y_train = [e["label"] for e in dataset["train"]]
input_pairs_val = [(e["sentence1"], e["sentence2"]) for e in dataset["validation"]]
y_val = [e["label"] for e in dataset["validation"]]
input_pairs_test = [(e["sentence1"], e["sentence2"]) for e in dataset["test"]]
y_test = [e["label"] for e in dataset["test"]]

In [40]:

def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = pipe(prepare(input), add_special_tokens=False)

    for prediction in predictions:
        prediction['score'] = logit(prediction['score'])

    y_pred = np.array([pred['score']for pred in predictions])
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.781988582023819


In [22]:
def compute_spearman(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = pipe(prepare(input), add_special_tokens=False)

    for prediction in predictions:
        prediction['score'] = logit(prediction['score'])

    y_pred = np.array([pred['score']for pred in predictions])
    # Calcular la correlación de Spearman entre las predicciones y los datos de prueba
    correlation, _ = spearmanr(y_pred.flatten(), np.array(y).flatten())
    return correlation

In [23]:
train_pearson = compute_pearson(input_pairs, y_train)
val_pearson = compute_pearson(input_pairs_val, y_val)
test_pearson = compute_pearson(input_pairs_test, y_test)

In [39]:
print(train_pearson, val_pearson, test_pearson)

0.9474290525726241 0.7522604466926754 0.781988582023819


In [24]:
train_spearman = compute_spearman(input_pairs, y_train)
val_spearman = compute_spearman(input_pairs_val, y_val)
test_spearman = compute_spearman(input_pairs_test, y_test)

In [25]:
print(train_spearman, val_spearman, test_spearman)

0.9614333116067086 0.7319414248170184 0.7993360636317496


## Our own finetuned

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the fine-tuned model and tokenizer
model_name = "pauhidalgoo/finetuned-sts-roberta-base-ca-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Use mean pooling to get sentence embeddings
    return embeddings

def calculate_similarity(sentence_pairs):
    similarities = []
    for s1, s2 in sentence_pairs:
        emb1 = get_embeddings([s1])
        emb2 = get_embeddings([s2])
        sim = cosine_similarity(emb1, emb2)[0][0]
        similarities.append(sim*5)
        
    return similarities

# Example sentence pairs
sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

# Calculate similarities
similarities = calculate_similarity(sentence_pairs)
print(similarities)


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2.063855677843094, 2.236715853214264, 0.8875995874404907]


In [73]:

def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = calculate_similarity(input)


    y_pred = np.array(predictions)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.742854611886582


Utilitzant el CLS

In [74]:
def get_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Use mean pooling to get sentence embeddings
    return embeddings

In [75]:

def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = calculate_similarity(input)


    y_pred = np.array(predictions)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.7146615222274435


In [9]:
a = TextSimilarity(model=model, dataset=dataset, mode="roberta-hugging", pretrained=False, cls=False, remap=True, trainable=True, tokenizer = tokenizer, dict_size=1000000, recalculate=True)

In [ ]:
a.define_model(id=5)

In [ ]:
pearson_train, pearson_val, pearson_test, _, _, _= a.train(num_epochs=128)